In [17]:
import psycopg2
import pandas as pd
import datetime
import shapely

In [28]:
digital_globe = pd.read_csv('../shared/imgs_metadata_2017.csv')
print digital_globe['footprintWkt']
cols = ['catalogID', 'timestamp', 'browseURL']
images = digital_globe[cols]

images['timestamp'] = images['timestamp'].values.astype('datetime64[s]')
#print images['timestamp'].min()
#print images['timestamp'].max()
#print images['']

0      MULTIPOLYGON(((142.1453524 -10.5029861, 142.36...
1      MULTIPOLYGON(((142.1531241 -10.80287813, 142.3...
2      MULTIPOLYGON(((142.2644451 -10.80582383, 142.4...
3      MULTIPOLYGON(((142.5044171 -9.294755032, 142.6...
4      MULTIPOLYGON(((142.3253557 -10.82007855, 142.4...
5      MULTIPOLYGON(((142.2778162 -10.87718206, 142.3...
6      MULTIPOLYGON(((142.1995686 -10.19949122, 142.3...
7      MULTIPOLYGON(((142.2145778 -10.13984084, 142.3...
8      MULTIPOLYGON(((142.2979277 -10.86376932, 142.3...
9      MULTIPOLYGON(((141.5406207 -8.979305446, 141.7...
10     MULTIPOLYGON(((142.9250632 -10.30107587, 142.7...
11     MULTIPOLYGON(((142.741833 -10.15742975, 142.90...
12     MULTIPOLYGON(((142.3944658 -10.11156505, 142.3...
13     MULTIPOLYGON(((143.6765772 -9.544605703, 143.8...
14     MULTIPOLYGON(((142.4970959 -9.286766287, 142.6...
15     MULTIPOLYGON(((142.3003942 -10.73137756, 142.4...
16     MULTIPOLYGON(((142.1105221 -10.10577578, 142.2...
17     MULTIPOLYGON(((142.78132

/opt/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [20]:
from configparser import ConfigParser


def config(filename='database.ini', section='postgresql'):
    
    parser = ConfigParser()
    parser.read(filename)
 
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))
 
    return db


def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    
    try:
        params = config()
 
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
 
        cur = conn.cursor()
        cur.execute("select timestamp_ais from ais_messages.spire_ais where timestamp_ais between '2017-01-02 03:45:29' and '2017-06-07 01:12:49'limit 5")
        rows = cur.fetchall()
        spire_ais = pd.DataFrame(rows)
        return spire_ais

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
 
 
if __name__ == '__main__':
    connect()

Connecting to the PostgreSQL database...
Database connection closed.


/opt/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  import sys


In [22]:
spire_ais = connect()
spire_ais.columns = ['msg_type', 'mmsi','timestamp', 'nmea', 'status', 'rot', 'speed', 
                     'accuracy', 'longitude', 'latitude', 'course', 'heading', 'manever']

print spire_ais
intersection = pd.merge(spire_ais, images, how='inner', on=['timestamp'])
#print intersection

Connecting to the PostgreSQL database...
Database connection closed.


/opt/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.
  import sys


ValueError: Length mismatch: Expected axis has 1 elements, new values have 13 elements